In [1]:
import nltk
import codecs
from nltk.stem import SnowballStemmer
stemmer = SnowballStemmer("english")

In [2]:
import os
from os.path import join
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english') + '. , ! ? ; - : —'.split())

In [3]:
DATA_DIR = join(os.environ['HOME'], 'data/allen-ai-challenge')

In [4]:
CORPUS = join(DATA_DIR, "ck12.txt")
WIKI_DIR = join(DATA_DIR, 'wiki_dump')

In [5]:
def tokenize(text):
    return [stemmer.stem(w) for w in nltk.word_tokenize(text.lower()) if w not in stopwords]

In [6]:
import json

In [7]:
text = ""

In [8]:
for i, fn_short in enumerate(os.listdir(WIKI_DIR)):
    fn = join(WIKI_DIR, fn_short)
    with codecs.open(fn, encoding='utf-8', errors='ignore') as f:
        wiki_article = json.load(f)
        _, summary, content = wiki_article
        text += summary + " " + content

In [9]:
len(text)

102542121

In [10]:
import telepot
def telegram_notify(msg):
    token = "178350795:AAFG7yae2SSt52GLek2bKS43oK7BaywWxRw"
    bot = telepot.Bot(token)
    b = bot.getMe()
    response = bot.getUpdates()
    bot.sendMessage(31747780, msg)

In [11]:
telegram_notify("read")

In [31]:
def iter_corpus():
    with codecs.open(CORPUS, encoding="utf-8") as f:
        for l in f:
            yield [stemmer.stem(w) for w in nltk.word_tokenize(l.lower()) if w not in stopwords]

In [12]:
len(text)

102542121

In [13]:
import nltk
from nltk.collocations import *
from nltk.tokenize import word_tokenize, sent_tokenize

# text = codecs.open(CORPUS, encoding="utf-8").read()

In [14]:
# trigram_measures = nltk.collocations.TrigramAssocMeasures()
# t_finder = TrigramCollocationFinder.from_words(tokenize(text))

In [19]:
%%time
bigram_measures = nltk.collocations.BigramAssocMeasures()
b_finder = BigramCollocationFinder.from_words(tokenize(text))

CPU times: user 6min 18s, sys: 7.06 s, total: 6min 25s
Wall time: 6min 31s


In [20]:
telegram_notify("bg")

In [21]:
pmi_scores = {}

In [22]:
b_finder.apply_freq_filter(3)

In [23]:
%%time
for i, t in enumerate(b_finder.score_ngrams(bigram_measures.pmi)):
    pmi_scores[t[0]] = t[1]

CPU times: user 6.44 s, sys: 92.4 ms, total: 6.54 s
Wall time: 6.71 s


In [24]:
len(pmi_scores)

679699

In [25]:
DATA_DIR = join(os.environ['HOME'], 'data/allen-ai-challenge')

In [26]:
TRAINING_SET = join(DATA_DIR, 'training_set.tsv')

In [27]:
for x in pmi_scores:
    print(x)
    break

(u'thought', u'sever')


In [28]:
pmi_scores[("a", "b")]

KeyError: ('a', 'b')

In [29]:
len(pmi_scores)

679699

In [30]:
def iter_data(datafile, with_correct=True):
    with codecs.open(datafile, encoding='utf-8', errors='ignore') as f:
        next(f)
        for l in f:
            if with_correct:
                idd, q, correct, aa, ab, ac, ad = l.strip().split("\t")
            else:
                idd, q, aa, ab, ac, ad = l.strip().split("\t")
                correct = "no"
            q, aa, ab, ac, ad = [' '.join(tokenize(x)) for x in [q, aa, ab, ac, ad]]
            yield {"idd": idd, "q": q, "correct": correct, "aa": aa, "ab": ab, "ac": ac, "ad": ad}

In [34]:
for x in pmi_scores:
    print(x)
    break

(u'thought', u'sever')


In [35]:
pmi_scores[('level', 'body')]

KeyError: ('level', 'body')

In [31]:
import numpy as np

In [33]:
tries = []
for i, row in enumerate(iter_data(TRAINING_SET)):
    answers = [row['aa'], row['ab'], row['ac'], row['ad']]
    q = row["q"]
    correct = row["correct"]
    scores = []
    for a in answers:
        pmis = []
        for wa in a.split():
            for wq in q.split():
                bg = (wa, wq)
                if bg in pmis:
                    pmis.append(pmi_scores[bg])
        scores = np.mean(pmis)
        print(scores)
    tries.append("ABCD"[np.argmax(scores)] == correct)
    print(q, correct, answers, scores)
    if i > 10:
        break

nan
nan
nan
nan
(u'athlet begin exercis heart rate respir rate increas level organ human bodi coordin function', u'C', [u'tissu level', u'organ level', u'system level', u'cellular level'], nan)
nan
nan
nan
nan
(u'exampl describ learn behavior dog', u'C', [u'smell air odor', u'bark disturb', u'sit command', u'dig soil'], nan)
nan
nan
nan
nan
(u'two nuclei combin one nucleus slight chang mass releas larg amount energi process call', u'D', [u'convers', u'reaction', u'fission', u'fusion'], nan)
nan
nan
nan
nan
(u'distinct epidem pandem', u'B', [u'symptom diseas', u'geograph area affect', u'speci organ infect', u'season diseas spread'], nan)
nan
nan
nan
nan
(u'way orbit comet differ orbit earth', u'B', [u'orbit earth less circular orbit comet', u'orbit comet ellipt orbit earth', u'orbit period earth much longer orbit period comet', u'orbit period comet predict orbit period earth'], nan)
nan
nan
nan
nan
(u'teacher build model hydrogen atom red golf ball use proton green golf ball use electro

In [142]:
np.mean(tries)

0.2336